In [6]:
#Copyright (c) 2019 Primoz Ravbar UCSB Licensed under BSD 2-Clause [see LICENSE for details] Written by Primoz Ravbar

import numpy as np
import scipy
from scipy import ndimage
from scipy import misc
import pickle
import time
import matplotlib.pyplot as plt
import cv2
import os

import tensorflow as tf
from tensorflow import keras

import ABRS_modules

from ABRS_modules import discrete_radon_transform
from ABRS_modules import create_ST_image
from ABRS_modules import smooth_2d
from ABRS_modules import center_of_gravity

from ABRS_modules import getting_frame_record
from ABRS_modules import read_frames

from ABRS_modules import subtract_average
from ABRS_modules import create_3C_image



def video_to_ethoProb_fun (dirPathInput,dirPathOutput,fbList,clipStart,clipEnd,clipsNumber,bufferSize,windowST,modelName,OSplatform='winds'):

    #OSplatform = 'winds'
    
    if modelName != 'none':
           
        model = keras.models.load_model(modelName)

    clips = np.arange(clipFirst,clipsNumber,1)

    fileList = os.listdir(dirPathInput)

    clIndex = 0

    for cl in clips:


        fileName = fileList[cl]

        ext = fileName[-3:]

        if (ext == 'avi' or ext == 'mov') == True:

            fileDirPathInputName = dirPathInput + '\\' + fileName

            r = np.arange(0,clipEnd,bufferSize)

            for bf in r:

                    startFrame = bf
                    endFrame = startFrame + bufferSize
                        
                    frRec = read_frames(startFrame, endFrame, fileDirPathInputName, [400,400])

                    if clIndex == clips[0] and startFrame == clipStart:
                        frRecRemain = np.zeros((windowST,160000));
                        frRec = np.concatenate((frRecRemain,frRec), axis=0)

                    if clIndex > clips[0] or startFrame > clipStart:
                        frRec = np.concatenate((frRecRemain,frRec), axis=0)  

                    for fb in fbList:

                        recIm3C = np.zeros((50,80,80,3))
                        predictionsProbRec = np.zeros((50,10))

                        for w in range(0,frRec.shape[0]-windowST):

                            startWin = w
                            endWin = startWin + windowST
                       
                            posDic, maxMovement, cfrVectRec, frameVectFloatRec = getting_frame_record(frRec, startWin, endWin,fb)


                            im3C = create_3C_image (cfrVectRec)
                            
                            
                            if modelName != 'none':
                                predictBehavior = 1  # predict behavior from ST-image and store the label
                            else:
                                predictBehavior = 0
                            
                            if predictBehavior == 1:
                                                                
                                X_rs = np.zeros((1,80,80,3))
        
                                X_rs[0,:,:,:]=im3C

                                #X = X_rs/1  # do not normalize #(ResNet)
                                X = X_rs/256  # normalize #(ResNet)

                                predictionsProb = model.predict(X)
                                predictionLabel = np.zeros((1,np.shape(predictionsProb)[0]))
                                predictionLabel[0,:] = np.argmax(predictionsProb,axis=1)
                    
                    
                            im_to_show = im3C
                        
                            label_text =  str(predictionLabel)[2:4]
                            #print(label_text)
                        
                            font = cv2.FONT_HERSHEY_SIMPLEX
  
                            # org
                            org = (50, 50)

                            # fontScale
                            fontScale = 1

                            # Blue color in BGR
                            color = (255, 0, 0)

                            # Line thickness of 2 px
                            thickness = 2
                                                
                            im_to_show = cv2.putText(im_to_show, label_text, org, font, fontScale, color, thickness, cv2.LINE_AA)
                    
                            cv2.imshow('im3C',im_to_show)
                        
                            #print(str(predictionLabel))
                        
                            #cv2.putText() 
                        
                            if cv2.waitKey(1) & 0xFF == ord('q'):
                                break

                            recIm3C[w,:,:,:]=im3C
                            #print(np.shape(recIm3C))
                            predictionsProbRec[w,:] = predictionsProb

                            xPos = posDic["xPos"]
                            yPos = posDic["yPos"]
                                                       

                            if w == 0:
                            
                                #sMRec = sMNorm;
                                xPosRec = xPos;
                                yPosRec = yPos;
                                maxMovementRec = maxMovement
                                    
                                if modelName != 'none': 
                                       behPredictionRec = predictionLabel 
                                else: behPredictionRec = 0        
                                
                            if w > 0:
                                
                                #sMRec = np.vstack((sMRec,sMNorm));
                                xPosRec = np.vstack((xPosRec,xPos));
                                yPosRec = np.vstack((yPosRec,yPos));
                                maxMovementRec = np.vstack((maxMovementRec,maxMovement))
                                
                                if modelName != 'none': 
                                    behPredictionRec = np.vstack((behPredictionRec,predictionLabel))
                                else: behPredictionRec = 0     
                                   
                        
                       
                        dictPosRec = {"xPosRec" : xPosRec, "yPosRec" : yPosRec};

                        #dictST = {"recIm3C" : recIm3C, "dictPosRec" : dictPosRec, "maxMovementRec" : maxMovementRec, "behPredictionRec" : behPredictionRec}
                        dictEtho = {"predictionsProbRec" : predictionsProbRec, "dictPosRec" : dictPosRec, "maxMovementRec" : maxMovementRec, "behPredictionRec" : behPredictionRec};
                        
                        #nameSMRec = 'dict3C_' + fileName[0:-4] + '_fb' + str(fb) + '_bf_' + str('%06.0f' % bf) 
                        nameSMRec = 'dictEtho_' + fileName[0:-4] + '_fb' + str(fb) + '_bf_' + str('%06.0f' % bf) 
                       
                        if OSplatform == 'winds':           
                            #newPath = dirPathOutput + '\\' + fileName[0:-4] + '_fb' + str(fb)  #!!!!!!works with non-numbered clips (one clip per video recording)
                            newPath = dirPathOutput + '\\' + fileName[0:-7] + '_fb' + str(fb) #!!!!!!works with numbered clips (multiple clips of the same video; clip names end with index number)

                        if OSplatform == 'mac':           
                            #newPath = dirPathOutput + '/' + fileName[0:-4] + '_fb' + str(fb)  #!!!!!!works with non-numbered clips (one clip per video recording)
                            newPath = dirPathOutput + '/' + fileName[0:-6] + '_fb' + str(fb) #!!!!!!works with numbered clips (multiple clips of the same video; clip names end with index number)
 

                        if not os.path.exists(newPath):
                            print('first_cond' + newPath)    
                            
                            if not os.path.exists(newPath):
                                print('second_cond' + newPath) 

                                print('new path:_' + newPath)
                                os.mkdir(newPath)
                                      
                        if OSplatform == 'winds': 
                            fileDirPathOutputName = newPath + '\\' + nameSMRec
                        if OSplatform == 'mac': 
                            fileDirPathOutputName = newPath + '/' + nameSMRec    
                                   
                        with open(fileDirPathOutputName, "wb") as f:
                            #pickle.dump(dictST, f)
                            pickle.dump(dictEtho, f)

                             
                    frRecSh = frRec.shape
                    frRecRemain = frRec[bufferSize:frRecSh[0],:]
            clIndex = clIndex +1
            print(clIndex)
                       
                
OSplatform = 'winds'
#OSplatform = 'mac';

frameRate = 60

clipStart = 0
clipEnd = 999 # number of frames in a clip to analyze

clipsNumberMax = 51 # when there are multiple clips in a folder

fbList = [1,2,3,4]; # works for raw movies with 2x2 arenas (split the frames into 4)
#fbList = [1]; # one arena in the frame

clipFirst = 0

bufferSize = 50

windowST = 16 #frame to be used for an ST-image; 16 works for 30-60 Hz videos

#modelName = 'model_ResNet__92.h5' #the name of the model
modelName = 'modelConv2ABRS_3C_train_with_descendingcombinedwithothers_avi_10' #the name of the model

firstFolder = 14 #  

if OSplatform == 'winds':
    
    # path to raw video data folder; video clips must be in subfolders at this path
    #rawVidDirPath = 'Path_to_videos' #set the path to the top input folder; each video should have its own subfolder
    rawVidDirPath = 'Z:\\primoz\\AVI_grooming_movies'
    
    
    # the output folder for ST images; subfolders will be created for each movie
    #dirPathOutput = 'Path_to_output_folder'


if OSplatform == 'mac':
    # path to raw video data folder; video clips must be in subfolders at this path
    rawVidDirPath = 'Path_to_videos' # path in Mac OS style

    # the output folder for ST images; subfolders will be created for each movie
    dirPathOutput = 'Path_to_output_folder'  # path in Mac OS style   

videoFolderList = sorted(os.listdir(rawVidDirPath))

sz = np.shape(videoFolderList);sizeVideoFolder = sz[0]

for fld in range(firstFolder, sizeVideoFolder):
    
    print(fld)

    currentVideoFolder = videoFolderList[fld]

    if currentVideoFolder[-5:] != 'Store' :

        dirPathInput = rawVidDirPath + '\\' + currentVideoFolder;
        clipList = sorted(os.listdir(dirPathInput));szClipList = np.shape(clipList);
        clipsNumber = szClipList[0];

        if clipsNumber > clipsNumberMax:
            clipsNumber = clipsNumberMax;

        video_to_ethoProb_fun (dirPathInput,dirPathOutput,fbList,clipStart,clipEnd,clipsNumber,bufferSize,windowST,modelName,OSplatform);


14
first_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb1
second_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb1
new path:_Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb1
first_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb2
second_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb2
new path:_Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb2
first_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb3
second_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb3
new path:_Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb3
first_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb4
second_condZ:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\CantonS_dust_1_fb4
new path:_Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50_new\Cant

KeyboardInterrupt: 

In [7]:
cv2.destroyAllWindows() 

In [14]:
print('new path:_' + newPath)

NameError: name 'newPath' is not defined

In [15]:
newPath = 'Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50\CantonS_dusted_13_fb4'

In [16]:
print('new path:_' + newPath)

new path:_Z:\primoz\ABRS_python_data\EthoProbJaneliaResNet50\CantonS_dusted_13_fb4


In [17]:
if not os.path.exists(newPath):
    print('new path:_' + newPath)

In [34]:
print(fld)

26
